In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pyLIA as lia
import pysfmov as sfmov

In [8]:
filename = './data/rec.sfmov'
fs = 400
(x,y,w,h) = 38,78,5,5 #vert mode area
k = 6.51 # slope endurance curve
B = 800.26 # endurance curve
km = 1/ (6 * 2 * 10**(-9))  # Calibration Factor [Pa^-1]

In [10]:

data = sfmov.get_data(filename)

In [12]:
nframes = data.shape[0]
dt = 1/fs
time = np.linspace(0,nframes/fs, nframes, endpoint=True)

bas = -1/k
C = B**k  # s = B * N**bas or s^k * N = C

In [13]:
temp_ampl = data - data[0,:,:]

In [14]:

stress_ampl = temp_ampl * km * 10**-6 # stress amplitude in MPa

# FFT

In [16]:
sig = stress_ampl
FFT = np.fft.rfft(sig,nframes-1,axis=0) *2/(nframes-1)
freq = np.fft.rfftfreq(nframes-1,1/fs)

FFT = FFT[:,y:(y+h),x:(x+w)]
FFT_avg = np.zeros((1,FFT.shape[0]))
for p in range(0,len(freq)):
    FFT_avg[0,p] = np.mean(np.abs(FFT[p,:,:]))

# Damage

In [18]:
band1 = (freq>=32)&(freq<=34)
band2 = (freq>=54)&(freq<=56)
A1 = np.mean(FFT_avg[0][band1])
A2 = np.mean(FFT_avg[0][band2])

C:\Users\Lorenzo\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Lorenzo\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [19]:
n1 = 33
n2 = 55
s1 = A1
s2 = A2
N1 = C / s1**k
N2 = C / s2**k
D1 = n1 / N1
D2 = n2 / N2
D = D1 + D2
D

nan

In [ ]:
#N = np.linspace(0,10**8,num = 10**8)
#s = B * N**bas

# %matplotlib notebook
# plt.semilogx(N,s)
# plt.grid()
# plt.xlabel('Cycles [Log(N)]')
# plt.ylabel('Stress amplitude [MPa]')
# plt.title('Endurance curve')